In [ ]:
#!pip install load_confounds

In [ ]:
import os
import pandas as pd
import numpy as np
import nilearn
import nibabel as nb

from nilearn.datasets import load_mni152_template
from nilearn.image import resample_to_img, load_img
from nilearn.plotting import plot_design_matrix
from nilearn.glm.first_level import FirstLevelModel, make_first_level_design_matrix
from nilearn import image, masking, plotting, maskers

#from load_resp_stim import *
import matplotlib.pyplot as plt

template = load_mni152_template()

### taken from trash/12_file_analysis_before_svm.ipynb

In [ ]:
def just_check_output_folder():
    parent_dir = "output/"
    subjects_to_not_consider = []
    for subject_dir in sorted(os.listdir(parent_dir)):
        subject_dir_path = os.path.join(parent_dir, subject_dir)
        if os.path.isdir(subject_dir_path) and "sub" in subject_dir_path:
            count = 0
            for visit_dir in sorted(os.listdir(subject_dir_path)):
                visit_dir_path = os.path.join(subject_dir_path, visit_dir)
                if os.path.isdir(visit_dir_path) and "ses" in visit_dir_path:
                    print(visit_dir_path)
                    count += 1
            if count < 4:
                subjects_to_not_consider.append(visit_dir_path.split("/")[1].split("-")[1])
            print("")
    return subjects_to_not_consider

In [ ]:
subjects_to_not_consider = just_check_output_folder() 
# this checks if all of the visits are in the output folder. If not, then the subject is not considered.

In [ ]:
print(subjects_to_not_consider) 

In [ ]:
def load_subject_sessions(subjects_to_not_consider):
    """This loads sessions from all subjects (everything in output), except the ones that are not to be considered."""

    parent_dir = "output/"
    
    subject_sessions = []
    for subject_dir in sorted(os.listdir(parent_dir)): # for each subject
        subject_dir_path = os.path.join(parent_dir, subject_dir) # get the path to the subject directory
        if os.path.isdir(subject_dir_path) and "sub" in subject_dir_path:  # make sure it's actually a subject directory, and that it's a directory
            for visit_dir in sorted(os.listdir(subject_dir_path)): # for every visit
                visit_dir_path = os.path.join(subject_dir_path, visit_dir) # get the path to the visit directory
                if os.path.isdir(visit_dir_path) and "ses" in visit_dir_path: # check it's a visit, and also check that it's a directory
                    if not any(subject_id in visit_dir_path for subject_id in subjects_to_not_consider): # if this is a subject to consider
                        subject_sessions.append(visit_dir_path) # then append it to the subject sessions list
    return subject_sessions

def load_subject_files(subject_sessions):
    df = pd.DataFrame()
    for session_dir in sorted(subject_sessions):
        dict_for_subject = {"subject": str(session_dir.split("/")[1][4:]), "session": str(session_dir.split("/")[2][-1])}
        for file in sorted(os.listdir(session_dir + "/func/")):
            if "confounds" in file:
                complete_file_path = os.path.join(session_dir + "/func/", file)
                if "mv" in file in file:
                    dict_for_subject["mv_confounds"] = complete_file_path
                elif "sp_run-01" in file:
                    dict_for_subject["sp_run_01_confounds"] = complete_file_path
                elif "sp_run-02" in file:
                    dict_for_subject["sp_run_02_confounds"] = complete_file_path
                elif "sv" in file:
                    dict_for_subject["sv_confounds"] = complete_file_path
                else:
                    pass
                
            if "nii.gz" in file:
                complete_file_path = os.path.join(session_dir + "/func/", file)
                if "mv" in file and "brain_mask" in file:
                    dict_for_subject["mv_brain_mask"] = complete_file_path
                elif "sp_run-01" in file and "brain_mask" in file:
                    dict_for_subject["sp_run_01_brain_mask"] = complete_file_path
                elif "sp_run-02" in file and "brain_mask" in file:
                    dict_for_subject["sp_run_02_brain_mask"] = complete_file_path
                elif "sv" in file and "brain_mask" in file:
                    dict_for_subject["sv_brain_mask"] = complete_file_path
                elif "mv" in file and "desc-preproc_bold" in file:
                    dict_for_subject["mv_bold"] = complete_file_path
                elif "sp_run-01" in file and "desc-preproc_bold" in file:
                    dict_for_subject["sp_run_01_bold"] = complete_file_path
                elif "sp_run-02" in file and "desc-preproc_bold" in file:
                    dict_for_subject["sp_run_02_bold"] = complete_file_path
                elif "sv" in file and "desc-preproc_bold" in file:
                    dict_for_subject["sv_bold"] = complete_file_path
                else:
                    pass
        new_row = pd.Series(dict_for_subject)
        df = pd.concat([df, new_row.to_frame().T], ignore_index=True)
    return df

In [ ]:
subject_sessions = load_subject_sessions(subjects_to_not_consider)
subject_files_df = load_subject_files(subject_sessions)

In [ ]:
subject_files_df # this is all the files, for all the subjects that we are considering

In [ ]:
subject_files_df.isna().sum()

The below code shows that one of the subjects is missing mv files. This means that for analysis involving the mv task, we must remove this participant. However, for the sp task, a different set of patients should be missed out (as we also need the response files).

In [ ]:
#subjects_to_not_consider.extend(subject_files_df.loc[subject_files_df['mv_bold'].isna()].subject.tolist())
print(subjects_to_not_consider)
subject_sessions = load_subject_sessions(subjects_to_not_consider)
subject_files_df = load_subject_files(subject_sessions)

In [ ]:
def check_nii_shapes(subject_files_df):
    """This function checks the shapes of all of the files in the sp_run_01_bold (mv is not relevant)"""
    
    sp_bold_files = subject_files_df["sp_run_01_bold"].tolist()
    subjects_to_not_consider = []

    for file in sp_bold_files:
        try:
            loaded_data = nb.load(file)
            if loaded_data.shape != (57, 68, 65, 244):
                print(file, loaded_data.shape)
                subjects_to_not_consider.append(file.split("/")[1].split("-")[1])
        except Exception as error:
            print(error)
            print(f"file: {file}")
    return list(set(subjects_to_not_consider))

In [ ]:
subjects_with_different_file_size = check_nii_shapes(subject_files_df)

In [ ]:
subjects_to_not_consider.extend(subjects_with_different_file_size)
print(subjects_to_not_consider)

In [ ]:
# Analysis from the file:
# 099 -> 2,3,4 sessions not present
# 104 -> 2,3,4 sessions not present
# 121 -> 4 session not present
# Reject subject 114, mv not available
# Reject sp_run_02, not available in most files
# Reject subject 098, some problems in the shape of output/sub-098/ses-visit4/func/sub-098_ses-visit4_task-sp_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz (57, 68, 60, 244)
# 098 isn't of shape (57, 68, 65, 244)

In [ ]:
subject_sessions = load_subject_sessions(subjects_to_not_consider)
subject_files_df = load_subject_files(subject_sessions)
#subject_files_df.to_excel("subject_files.xlsx")

In [ ]:
subject_files_df

In [ ]:
subject_files_df.isna().sum()

Clean files before creating model

https://brainhack-princeton.github.io/handbook/content_pages/05-01-univariate.html

https://github.com/davide-aloi/raindrop_analyses_fmri_eeg/blob/a2de41fb6057a73f4eb029c301fcff00710c5157/fmri/preprocessing/resting-state/rs_preprocessing_part_2.ipynb

https://nilearn.github.io/dev/auto_examples/04_glm_first_level/plot_predictions_residuals.html#sphx-glr-auto-examples-04-glm-first-level-plot-predictions-residuals-py

https://carpentries-incubator.github.io/SDC-BIDS-fMRI/05-data-cleaning-with-nilearn/index.html

https://neurostars.org/t/fmriprep-appling-ica-aroma-and-other-regressors/919/3

https://neurostars.org/t/including-fmriprep-a-comp-cor-components-causes-raw-and-cleaned-data-to-be-anti-correlated/22884

https://github.com/danjgale/psyc-917/tree/9ac6414cdb878a18f2db6e9e9a2a8752e64b0c0a

IMPORTANT: (an explanation of different confounds) https://fcp-indi.github.io/docs/latest/user/nuisance

```
six parameters obtained by rigid body correction of head motion, []
the whole-brain signal averaged over all voxels of the brain, ['global_signal', 'global_signal_derivative1', 'global_signal_derivative1_power2', 'global_signal_power2']
signal from a ventricular ROI and signal from a region centered in the white matter. ['csf', 'csf_derivative1', 'csf_derivative1_power2', 'csf_power2', 'white_matter', 'white_matter_derivative1', 'white_matter_power2', 'white_matter_derivative1_power2', 'csf_wm']
```



In [ ]:
def clean_images(df):
    """This function cleans all of the images in the subject dataframe. Once again, mv tasks are not relevant.
    It then saves the cleaned images to the output_cleaned directory"""

    clean_images_path_sp_run_01_bold = []
    
    output_dir = "output_cleaned/"
    try:
        os.mkdir(output_dir)
    except:
        pass
    
    for index, row in df.iterrows():
        """"
        print(row["mv_bold"])
        print(row["mv_confounds"])
        
        print(row["sp_run_01_bold"])
        print(row["sp_run_01_confounds"])
        """
        
        folder_output_path = os.path.join(output_dir, os.path.join(*row["mv_bold"].split("/")[1:-1]))
        mv_bold_output_path = os.path.join(output_dir, os.path.join(*row["mv_bold"].split("/")[1:]))
        sp_run_01_bold_output_path = os.path.join(output_dir, os.path.join(*row["sp_run_01_bold"].split("/")[1:]))
        print(folder_output_path, mv_bold_output_path, sp_run_01_bold_output_path, sep="\n")
        
        """"
        fmri_img = image.clean_img(file, confounds=confounds, detrend=True, standardize=True,
                                   low_pass=low_pass, high_pass=high_pass, t_r=t_r)
        fmri_img = image.smooth_img(fmri_img, 5.)
        fmri_img.to_filename()
        """
    return None

In [ ]:
subject = "097"
session = "4"
task = "sp_run_01"
subject_files_df.loc[(subject_files_df.subject == subject) & (subject_files_df.session == session)][task + "_bold"].tolist()[0]

In [ ]:
subject_files_df.loc[(subject_files_df.subject == subject) & (subject_files_df.session == session)]

In [ ]:
session = "4"
subject_files_df.loc[(subject_files_df.subject == subject) & (subject_files_df.session == session)]["sp_run_01_bold"].tolist()

## 3/11/22 - Work on creating this:

https://nilearn.github.io/stable/auto_examples/04_glm_first_level/plot_predictions_residuals.html#sphx-glr-auto-examples-04-glm-first-level-plot-predictions-residuals-py

In [ ]:
def preprocess_file(subject_id = "097", task = "sp_run_01", session = "1"):
    """This function runs the preprocessing for one subject, task and session."""

    from load_confounds import Confounds
    
    #index the subject_files_df for a subject, taks and session (task uses different syntax here)
    bold_file = subject_files_df.loc[(subject_files_df.subject == subject_id) & (subject_files_df.session == session)][task + "_bold"].tolist()[0]
    mask_file = subject_files_df.loc[(subject_files_df.subject == subject_id) & (subject_files_df.session == session)][task + "_brain_mask"].tolist()[0]
    
    confounds = Confounds(strategy=["high_pass", "motion", "global", "wm_csf"], motion="basic", global_signal="basic", wm_csf="basic").load(bold_file)
    print(confounds.shape)
    
    t_r = 2.5
    high_pass= 0.006
    low_pass = None
    bold_img = nilearn.image.load_img(bold_file)
    bold_img = bold_img.slicer[:,:,:,:]
    plotting.plot_epi(bold_img.slicer[:, :, :, 50])
    confounds_matrix = confounds[:]
    cleaned_bold = nilearn.image.clean_img(bold_img, 
                                              confounds=confounds_matrix, 
                                              detrend=False, 
                                              standardize=False, 
                                              low_pass=low_pass, 
                                              high_pass=high_pass,
                                              ensure_finite=True,
                                              mask_img=mask_file,
                                              t_r = t_r
                                             )
    cleaned_bold = nilearn.image.smooth_img(cleaned_bold, fwhm=7)
    plotting.plot_epi(cleaned_bold.slicer[:, :, :, 50])
    return cleaned_bold, mask_file

In [ ]:
def high_and_low(subject_id = "097", session = "1", stat_threshold = 2, cluster_threshold = 1):
    """This function performs a GLM on a single subject, for a single session, using the sp_run-01 task.
    It contrasts overall high activation against overall low activation.
    """

    # Load the sp bold, mask and response
    sp_bold, sp_mask_file = preprocess_file(subject_id = subject_id, task = "sp_run_01", session = session)
    sp_downsampled_response = load_sp_func(participant_id= subject_id, visit= session, run= 1)

    # Create a design matrix, from a dataframe
    df_events = pd.DataFrame()
    df_events["modulation"] = sp_downsampled_response[4:]
    df_events["onset"] = df_events.index
    df_events["duration"] = 1
    modulation_mean = df_events["modulation"].mean()

    df_events["trial_type"] = df_events["modulation"].apply(lambda x: 'low' if x < modulation_mean else 'high')

    design_matrices = make_first_level_design_matrix(df_events.index, df_events, drift_model='polynomial', drift_order=3, hrf_model="glover")
    plot_design_matrix(design_matrices)

    # Create the First Level Model
    fmri_glm = FirstLevelModel(minimize_memory=False, mask_img=sp_mask_file)
    fmri_glm = fmri_glm.fit(sp_bold, design_matrices=design_matrices)
    mean_img = image.mean_img(sp_bold)

    #compute the contrast, recovering z values
    z_map = fmri_glm.compute_contrast('high - low')
    plotting.plot_stat_map(z_map, bg_img=mean_img, threshold = stat_threshold)
    
    design_matrix = fmri_glm.design_matrices_[0]
    # contrast with a one for "active" and zero everywhere else
    high = np.array([1 if c == 'high' else 0 for c in design_matrix.columns])
    # contrast with a one for "rest" and zero everywhere else
    low = np.array([1 if c == 'low' else 0 for c in design_matrix.columns])
    effects_of_interest = np.vstack((high, low))
    # f-test for rest and activity
    z_map_ftest = fmri_glm.compute_contrast(effects_of_interest, stat_type='F', output_type='z_score')
    plotting.plot_stat_map(z_map_ftest, bg_img=mean_img, threshold = stat_threshold, display_mode='z', cut_coords=7)

def norm(x):
    return (x - np.mean(x))/np.std(x)

from scipy.signal import savgol_filter

def run_modulation_GLM(subject_id = "100", session = "1", stat_threshold = 2, cluster_threshold = 1):
    """Runs a GLM to find neural correlates between the filtered responses for a single subject's sp task, for one session."""

    sp_bold, mask_file_sp = preprocess_file(subject_id = subject_id, task = "sp_run_01", session = session)
    run  = 'sp_run1'
    # TODO: CHANGE THE WAY WE LOAD THE RESPONSES
    # Get the filtered responses
    filteredResponsePath = os.path.join('Carl_RL_filtered_responses_sp',f'sub-{subject_id}',f'visit{session}',run,'rl_filtered_responses.npz')
    filteredResponses = np.load(filteredResponsePath, allow_pickle=True)

    # Split out the regressors
    RESPONSE_LENGTH = 8784
    BURN_IN = 4*36
    original_responses = filteredResponses['response_to_filter'][BURN_IN:RESPONSE_LENGTH]
    times = np.arange(len(original_responses))
    errors = filteredResponses['errors'][BURN_IN:RESPONSE_LENGTH]
    expectations = filteredResponses['expectations'][BURN_IN:RESPONSE_LENGTH]

    fig, ax = plt.subplots(3,figsize=(20,5))
    ax[0].plot(times, original_responses,label = 'original responses')
    ax[0].legend()
    ax[1].plot(times, errors, label = 'errors')
    ax[1].legend()
    ax[2].plot(times, expectations, label = 'expectations')
    ax[2].legend()
    plt.legend()
    plt.show()

    # TODO Downsample the responses, work out when the responses should be clipped *before convolution?*
    downsampled_original_responses = original_responses[::36]
    downsampled_errors = errors[::36]
    downsampled_expectations = expectations[::36]
    # TODO Ask Flavia whether I should smooth the responses as well, to account for movement inaccuracies from the subject

    #convolve the errors
    hrf_8784 = nilearn.glm.first_level.glover_hrf(2.5, oversampling=36, onset = 0.0)
    convolved_errors = np.convolve(norm(errors),hrf_8784)[:len(errors)][::36].reshape(-1,1)

    # No need for this, as the pre and post subsampling convolutions are practically identical
    #convolved_expectations = np.convolve(norm(expectations),hrf_8784)[:len(errors)][::36].reshape(-1,1)
    

    # Create the dataframe # TODO check these
    df_events = pd.DataFrame()
    df_events["modulation"] = norm(downsampled_original_responses) # TODO check mean centering
    df_events["onset"] = np.arange(len(convolved_errors)) * 2.5
    df_events["duration"] = 2.5 # this should be TR ( the duration of each event in seconds)
    df_events["trial_type"] = 'response' # this should be TR ( the duration of each event in seconds)
    design_matrices = make_first_level_design_matrix(
        frame_times = df_events.index * 2.5,
        events = df_events, 
        drift_model='polynomial', 
        drift_order=3, 
        hrf_model="glover",
        add_regs= convolved_errors,
        add_reg_names= ['errors']
    )

    font = 25

    fig = plt.figure(figsize = (20,5))
    plt.plot(np.arange(len(design_matrices['response'])),norm(design_matrices['response']), label='normalised convolved expectations')
    plt.plot(np.arange(len(design_matrices['response'])) ,norm(design_matrices['errors']),label='normalised convolved errors')
    plt.plot(np.arange(36*len(design_matrices['response']))/36.0 ,norm(expectations),label='original_response', linewidth = 0.5)
    #plt.plot(np.arange(36*len(design_matrices['dummy']))/36.0 ,errors,label='original errors', linewidth = 0.5)
    plt.xticks(np.arange(244)[::16],np.array(np.arange(0,244)[::16]*2.5).astype(int),rotation=90,fontsize=font)
    plt.yticks(fontsize=font)
    plt.xlabel("seconds", fontsize=font)
    plt.legend(fontsize=font/2)
    plt.ylabel("normalised regressor \n magnitude", fontsize=font)
    plt.suptitle("The convolved regressors for use in the GLM", fontsize=font)
    plt.show()

    plot_design_matrix(design_matrices)

    # Create a first level model with 
    fmri_glm = FirstLevelModel(minimize_memory=False, mask_img=mask_file_sp)
    fmri_glm = fmri_glm.fit(sp_bold, design_matrices=design_matrices)

    # something like this
    #z_map_ftest = fmri_glm.compute_contrast('errors', stat_type='T', output_type='z_score')

    # mean_img = image.mean_img(sp_bold)
    # mean_residuals = image.mean_img(fmri_glm.residuals)
    # plotting.plot_glass_brain(mean_residuals, colorbar=True)
    # plotting.plot_epi(mean_residuals, colorbar=True)

# Change to the GLM for a report plot ONLY

In [ ]:
def norm(x):
    return (x - np.mean(x))/np.std(x)

def run_modulation_GLM(subject_id = "100", session = "1", stat_threshold = 2, cluster_threshold = 1):
    """Runs a GLM to find neural correlates between the filtered responses for a single subject's sp task, for one session."""

    sp_bold, mask_file_sp = preprocess_file(subject_id = subject_id, task = "sp_run_01", session = session)
    run  = 'sp_run1'
    # TODO: CHANGE THE WAY WE LOAD THE RESPONSES
    # Get the filtered responses
    filteredResponsePath = os.path.join('Carl_RL_filtered_responses_sp',f'sub-{subject_id}',f'visit{session}',run,'rl_filtered_responses.npz')
    filteredResponses = np.load(filteredResponsePath, allow_pickle=True)

    # Split out the regressors
    RESPONSE_LENGTH = 8784
    BURN_IN = 4*36
    original_responses = filteredResponses['response_to_filter'][BURN_IN:RESPONSE_LENGTH]
    times = np.arange(len(original_responses))
    errors = filteredResponses['errors'][BURN_IN:RESPONSE_LENGTH]
    expectations = filteredResponses['expectations'][BURN_IN:RESPONSE_LENGTH]

    fig, ax = plt.subplots(3,figsize=(20,5))
    ax[0].plot(times, original_responses,label = 'original responses')
    ax[0].legend()
    ax[1].plot(times, errors, label = 'errors')
    ax[1].legend()
    ax[2].plot(times, expectations, label = 'expectations')
    ax[2].legend()
    plt.legend()
    plt.show()

    # TODO Downsample the responses, work out when the responses should be clipped *before convolution?*
    downsampled_original_responses = original_responses[::36]
    downsampled_errors = errors[::36]
    downsampled_expectations = expectations[::36]
    # TODO Ask Flavia whether I should smooth the responses as well, to account for movement inaccuracies from the subject

    #convolve the errors
    hrf_8784 = nilearn.glm.first_level.glover_hrf(2.5, oversampling=36, onset = 0.0)
    convolved_errors = np.convolve(norm(errors),hrf_8784)[:len(errors)][::36].reshape(-1,1)

    # No need for this, as the pre and post subsampling convolutions are practically identical
    #convolved_expectations = np.convolve(norm(expectations),hrf_8784)[:len(errors)][::36].reshape(-1,1)
    

    # Create the dataframe # TODO check these
    df_events = pd.DataFrame()
    df_events["modulation"] = norm(downsampled_original_responses) # TODO check mean centering
    df_events["onset"] = np.arange(len(convolved_errors)) * 2.5
    df_events["duration"] = 2.5 # this should be TR ( the duration of each event in seconds)
    df_events["trial_type"] = 'response' # this should be TR ( the duration of each event in seconds)
    design_matrices = make_first_level_design_matrix(
        frame_times = df_events.index * 2.5,
        events = df_events, 
        drift_model='polynomial', 
        drift_order=3, 
        hrf_model="glover",
        add_regs= convolved_errors,
        add_reg_names= ['errors']
    )

    font = 25

    fig,axes = plt.subplots(2,1,figsize = (16,5),sharex=True)
    ax = axes.flatten()
    ax[0].plot(np.arange(len(design_matrices['response'])),norm(design_matrices['response']), label='predictions')
    ax[0].plot(np.arange(len(design_matrices['response'])) ,norm(design_matrices['errors']),label='errors')
    #ax[1].plot(np.arange(36*len(design_matrices['response']))/36.0 ,norm(expectations),label='original_response')
    ax[1].plot(np.arange(36*len(design_matrices['response']))/36.0 ,savgol_filter(norm(expectations),8*36+1,3), label='original_response')

    ax[0].tick_params(axis='y', labelsize=font/1.5)
    ax[1].tick_params(axis='y', labelsize=font/1.5)
    plt.xticks(np.arange(244)[::16],np.array(np.arange(0,244)[::16]*2.5).astype(int),rotation=90,fontsize=font/1.5)

    ax[1].legend(fontsize=font/1.75,loc='upper left')
    ax[0].legend(fontsize=font/1.75, loc='upper left')
    plt.xlabel("seconds", fontsize=font)
    fig.add_subplot(111, frameon=False)
    # hide tick and tick label of the big axis
    plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
    plt.ylabel("Normalised \n magnitude", fontsize=font)    
    plt.suptitle("Convolved RL regressors for use in the GLM, and the smoothed pain ratings", fontsize=font)
    plt.tight_layout()
    plt.show()

    plot_design_matrix(design_matrices)

    # Create a first level model with 
    fmri_glm = FirstLevelModel(minimize_memory=False, mask_img=mask_file_sp)
    fmri_glm = fmri_glm.fit(sp_bold, design_matrices=design_matrices)

    # something like this
    #z_map_ftest = fmri_glm.compute_contrast('errors', stat_type='T', output_type='z_score')

    # mean_img = image.mean_img(sp_bold)
    # mean_residuals = image.mean_img(fmri_glm.residuals)
    # plotting.plot_glass_brain(mean_residuals, colorbar=True)
    # plotting.plot_epi(mean_residuals, colorbar=True)

In [ ]:
#run_modulation_GLM()

clean_images(subject_files_df)

In [ ]:
def load_shape_and_affine(object_passed):
    if type(object_passed) == str:
        img = nb.load(object_passed)
    else:
        img = object_passed
    return img.shape, img.affine

def resample_all_data(df):
    new_df = df.copy(deep=True)
    
    sp_run_01_bold_resampled = []
    
    orginal_shape_sp_run_01_bold = []
    resampled_shape_sp_run_01_bold = []
    
    original_affine_sp_run_01_bold = []
    resampled_affine_sp_run_01_bold = []

    for index, row in df.iterrows():

            print(row["sp_run_01_bold"])

            folder_output_path = os.path.join("output_resampled/", os.path.join(*row["mv_bold"].split("/")[1:-1]))
            sp_run_01_bold_output_path = os.path.join("output_resampled/", os.path.join(*row["sp_run_01_bold"].split("/")[1:]))

            print(folder_output_path, sp_run_01_bold_output_path, sep="\n")
            
            try:
                os.makedirs(folder_output_path)
            except:
                pass
                        
            resampled_sp_run_01_bold = resample_to_img(row["sp_run_01_bold"], template)
            resampled_sp_run_01_bold.to_filename(sp_run_01_bold_output_path)
            sp_run_01_bold_resampled.append(sp_run_01_bold_output_path)
            orginal_shape_sp_run_01_bold.append(load_shape_and_affine(row["sp_run_01_bold"])[0])
            original_affine_sp_run_01_bold.append(load_shape_and_affine(row["sp_run_01_bold"])[1])
            resampled_shape_sp_run_01_bold.append(load_shape_and_affine(resampled_sp_run_01_bold)[0])
            resampled_affine_sp_run_01_bold.append(load_shape_and_affine(resampled_sp_run_01_bold)[1])
            
    new_df["sp_run_01_bold_resampled"] = sp_run_01_bold_resampled
    
    new_df["orginal_shape_sp_run_01_bold"] = orginal_shape_sp_run_01_bold
    new_df["resampled_shape_sp_run_01_bold"] = resampled_shape_sp_run_01_bold
    
    new_df["original_affine_sp_run_01_bold"] = original_affine_sp_run_01_bold
    new_df["resampled_affine_sp_run_01_bold"] = resampled_affine_sp_run_01_bold
    
    return new_df

def resample_all_brain_masks(df):
    new_df = df.copy(deep=True)
    
    sp_run_01_brain_mask_resampled = []
    
    for index, row in df.iterrows():
        print(row["sp_run_01_brain_mask"])
        folder_output_path = os.path.join("output_resampled/", os.path.join(*row["mv_brain_mask"].split("/")[1:-1]))
        sp_run_01_brain_mask_output_path = os.path.join("output_resampled/", os.path.join(*row["sp_run_01_brain_mask"].split("/")[1:]))
        print(folder_output_path, sp_run_01_brain_mask_output_path)
        try:
            os.makedirs(folder_output_path)
        except:
            pass
        
        resampled_sp_run_01_brain_mask = resample_to_img(row["sp_run_01_brain_mask"], template, interpolation = 'nearest')
        resampled_sp_run_01_brain_mask.to_filename(sp_run_01_brain_mask_output_path)
        sp_run_01_brain_mask_resampled.append(sp_run_01_brain_mask_output_path)
    
    new_df["sp_run_01_brain_mask_resampled"] = sp_run_01_brain_mask_resampled
    return new_df